# **Waste Material Segregation for Improving Waste Management**

## **Objective**

The objective of this project is to implement an effective waste material segregation system using convolutional neural networks (CNNs) that categorises waste into distinct groups. This process enhances recycling efficiency, minimises environmental pollution, and promotes sustainable waste management practices.

The key goals are:

* Accurately classify waste materials into categories like cardboard, glass, paper, and plastic.
* Improve waste segregation efficiency to support recycling and reduce landfill waste.
* Understand the properties of different waste materials to optimise sorting methods for sustainability.

## **Data Understanding**

The Dataset consists of images of some common waste materials.

1. Food Waste
2. Metal
3. Paper
4. Plastic
5. Other
6. Cardboard
7. Glass


**Data Description**

* The dataset consists of multiple folders, each representing a specific class, such as `Cardboard`, `Food_Waste`, and `Metal`.
* Within each folder, there are images of objects that belong to that category.
* However, these items are not further subcategorised. <br> For instance, the `Food_Waste` folder may contain images of items like coffee grounds, teabags, and fruit peels, without explicitly stating that they are actually coffee grounds or teabags.

## **1. Load the data**

Load and unzip the dataset zip file.

**Import Necessary Libraries**

In [ ]:
# Recommended versions:

# numpy version: 1.26.4
# pandas version: 2.2.2
# seaborn version: 0.13.2
# matplotlib version: 3.10.0
# PIL version: 11.1.0
# tensorflow version: 2.18.0
# keras version: 3.8.0
# sklearn version: 1.6.1

In [ ]:
# Import essential libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib 
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import keras
import sklearn
import zipfile
import os
import warnings 

warnings.filterwarnings('ignore')

# Pandas by default doesn't display all the columns in the dataframe
# As we're going to work on a large dataset, the following setting will help read data from all the columns
pd.set_option('display.max_columns', None)

# Disable scientific notation
pd.options.display.float_format = '{:,.2f}'.format

# For some of the columns we may have to see the data from all rows
# Eg: Categorical columns.
pd.set_option('display.max_rows', None)

# Set themes, styles and color palette for seaborn charts
sns.set_theme(style='darkgrid', context='poster')
sns.set_palette(palette='pastel', n_colors=10)

plt.rcParams["figure.figsize"] = (12, 4)
plt.rcParams["axes.titlesize"] = 14 
plt.rcParams["axes.labelsize"] = 12 
plt.rcParams["xtick.labelsize"] = 10 
plt.rcParams["ytick.labelsize"] = 10 

# Print version numbers
print('Numpy version:      ' + np.__version__)
print('Pandas version:     ' + pd.__version__)
print('Seaborn version:    ' + sns.__version__)
print('Matplotlib version: ' + matplotlib.__version__)
print('PIL version:        ' + Image.__version__)
print('TensorFlow version: ' + tf.__version__)
print('Keras version:      ' + keras.__version__)
print('Scikit-learn version: ' + sklearn.__version__)

Load the dataset.

In [ ]:
# Load and unzip the dataset

# The data.zip is loaded from https://drive.google.com/drive/folders/1sajIcvGxBemqK_YIHFoY28EyV1Su_b5M?usp=drive_link
zip_path = 'data.zip' 
extract_folder = 'waste_dataset' 

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f"Dataset extracted to '{extract_folder}'")

## **2. Data Preparation** <font color=red> [25 marks] </font><br>


### **2.1 Load and Preprocess Images** <font color=red> [8 marks] </font><br>

Let us create a function to load the images first. We can then directly use this function while loading images of the different categories to load and crop them in a single step.

#### **2.1.1** <font color=red> [3 marks] </font><br>
Create a function to load the images.

In [ ]:
# Create a function to load the raw images
def load_images(folder_name, target_size=(64, 64)):
    images = []
    labels = []
    # sub_folder_names = os.listdir(folder_name)
    sub_folder_names = [name for name in os.listdir(folder_name) if name != 'Other']

    for sub_folder_name in sub_folder_names:
        sub_folder_path = os.path.join(folder_name, sub_folder_name)

        file_names = os.listdir(sub_folder_path)

        for file_name in filter(lambda k: '.png' in k, file_names):
            file_path = os.path.join(sub_folder_path, file_name)

            try:
                file = Image.open(file_path).convert(mode='RGB')
                file = file.resize(target_size)

                normalized_image_array = np.array(file, dtype=np.float32) / 255.0
                
                images.append(normalized_image_array)
                labels.append(sub_folder_name)
            except Exception as ex:
                print(f"Error loading image {file_path}: {ex}")
    
    return np.array(images), np.array(labels)


#### **2.1.2** <font color=red> [5 marks] </font><br>
Load images and labels.

Load the images from the dataset directory. Labels of images are present in the subdirectories.

Verify if the images and labels are loaded correctly.

In [ ]:
# Get the images and their labels
extract_folder_data = os.path.join(extract_folder, 'data')

X, y = load_images(extract_folder_data)

In [ ]:
print(f"Number of images loaded: {len(X)}")
print(f"Number of labels: {len(y)}")
print(f"Unique labels: {np.unique(y)}")

# Quick Observation
# ------------------------
# 1. There are 7625 images loaded and 7 unique labels
# ------------------------

Perform any operations, if needed, on the images and labels to get them into the desired format.

### **2.2 Data Visualisation** <font color=red> [9 marks] </font><br>

#### **2.2.1** <font color=red> [3 marks] </font><br>
Create a bar plot to display the class distribution

In [ ]:
# Visualise Data Distribution
label_counts = pd.Series(y).value_counts()

sns.barplot(x=label_counts.index, y=label_counts.values)
plt.title('Class Distribution of Waste Categories')
plt.xlabel('Waste Category')
plt.ylabel('Number of Images')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### **2.2.2** <font color=red> [3 marks] </font><br>
Visualise some sample images

In [ ]:
# Visualise Sample Images (across different labels)
import random

combined_data = list(zip(X, y))
unique_labels = np.unique(y)
sample_images = [random.choice([img for img, label in combined_data if label == l]) for l in unique_labels]

plt.figure(figsize=(14, 4))

for i, img in enumerate(sample_images):
    plt.subplot(2, (len(sample_images) + 1) // 2, i + 1)
    plt.imshow(img)
    plt.title(unique_labels[i])
    plt.axis('off')

plt.tight_layout()
plt.show()

#### **2.2.3** <font color=red> [3 marks] </font><br>
Based on the smallest and largest image dimensions, resize the images.

In [ ]:
# Find the smallest and largest image dimensions from the data set

# 1. I have already resized and normalized the images. Hence showing the image size of hte sample imageset.

for label, img in zip(unique_labels, sample_images):
    h, w = img.shape[:2]
    print(f"Label: {label}, Height: {h}, Width: {w}")

# Quick Observation
# ------------------------
# 1. The height and width of the images are 64.
#    As you can notice hte signature of function load_images(folder_name, target_size=(64, 64)).
#    The attribute target_size is defaulted to 64, 64
#    Hence the images are resized before being loaded into the collection.
# ------------------------

### **2.3 Encoding the classes** <font color=red> [3 marks] </font><br>

There are seven classes present in the data.

We have extracted the images and their labels, and visualised their distribution. Now, we need to perform encoding on the labels. Encode the labels suitably.

####**2.3.1** <font color=red> [3 marks] </font><br>
Encode the target class labels.

In [ ]:
# Encode the labels suitably
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

encoded_labels = label_encoder.fit_transform(y)

for id, label in enumerate(label_encoder.classes_):
    print(f"{id}: {label}")

print("Encoded collection:", encoded_labels)
print("Actual collection:", y)

### **2.4 Data Splitting** <font color=red> [5 marks] </font><br>

#### **2.4.1** <font color=red> [5 marks] </font><br>
Split the dataset into training and validation sets

In [ ]:
# Assign specified parts of the dataset to train and validation sets
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, encoded_labels, test_size=0.3, stratify=encoded_labels, random_state=100)

print(f"Image collection training set: {X_train.shape}")
print(f"Image collection validation set: {X_val.shape}")

print(f"Image labels training set: {y_train.shape}")
print(f"Image labels validation set: {y_val.shape}")

In [ ]:
unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_val, counts_val = np.unique(y_val, return_counts=True)

print("Training set class distribution:", dict(zip(unique_train, counts_train)))
print("Validation set class distribution:", dict(zip(unique_val, counts_val)))

## **3. Model Building and Evaluation** <font color=red> [20 marks] </font><br>

### **3.1 Model building and training** <font color=red> [15 marks] </font><br>

#### **3.1.1** <font color=red> [10 marks] </font><br>
Build and compile the model. Use 3 convolutional layers. Add suitable normalisation, dropout, and fully connected layers to the model.

Test out different configurations and report the results in conclusions.

In [ ]:
# Build and compile the model

# We'll use sequential type of model. In this neural layers are added in a linear stack, one after the other
# In our case we'll have an input layer, 3 convolutional layers and an output layer
from tensorflow.keras.models import Sequential

# Conv2D is used for applying filters and detecting patterns
# MaxPooling2D is used to impliment the pooling concept ie., get a 2 x 2 region and get the max value
# Flatten is used to convert 2D feature maps to single dimension array, which can be fed into dense layers. 6 x 6 pixel image to an array of len 36
# Dense takes the flatten input from each neuron, applies and activation function and outputs probabilities for each class.
# Dropout drops a fraction of the neurons during training, helps prevent overfitting.
# BatchNormalization helps in speeding up learning
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input)

In [ ]:
input_shape = sample_images[0].shape
num_classes = len(unique_labels)

# Reference: https://keras.io/api/models/sequential/
# Sequential will group a linear stack of layers into a model. We can add multiple layers to the stack, one at a time.
model = Sequential()

model.add(Input(shape=input_shape))

In [ ]:
# Convolutional layer 1 added to the model
# ----------------------------------------
# Reference: https://keras.io/api/layers/convolution_layers/convolution2d/
# 32 filters in the convolution
# The convolutional window is 3,3
# We'll use relu activation function
# We'll use default stride 1, 1 ie., the filter will move one step at a time. Padding 'same' will add padding on all sides to retain the shape
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))

# Reference: https://keras.io/api/layers/normalization_layers/batch_normalization/
# Batch normalization applies a transformation that maintains the mean output close to 0 and the output standard deviation close to 1.
model.add(BatchNormalization())

# Reference: https://keras.io/api/layers/pooling_layers/max_pooling2d/
# By doing pooling, the system will go through 2 x 2 matrix and get the max value.
model.add(MaxPooling2D(pool_size=(2, 2)))

# Reference: https://keras.io/api/layers/regularization_layers/dropout/
# Dropout is a regularization technique. In the following code we're turning off 25% of the neurons
model.add(Dropout(0.25))
# ----------------------------------------

In [ ]:
# Convolutional layer 2 added to the model
# ----------------------------------------
# The number of filters is increased to 64 for deeper learning than the basic understanding of the image
# Also we're dropping 25% more neurons
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# ----------------------------------------

In [ ]:
# Convolutional layer 3 added to the model
# ----------------------------------------
# The number of filters is increased to 128 for deeper learning than the basic understanding of the image
# Also we're dropping 25% more neurons
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# ----------------------------------------

In [ ]:
# Fully connected layers
model.add(Flatten())  
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# Reference: https://keras.io/api/models/model_training_apis/
# Compile the model
# Because labels are integer encoded, we use the loss function sparse_categorical_crossentropy https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class
model.compile(
    # optimizer='adam',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
# Print shape details
print("Layer-by-layer output shapes:\n")
for i, layer in enumerate(model.layers):
    print(layer)
    print(layer.output.shape)

#### **3.1.2** <font color=red> [5 marks] </font><br>
Train the model.

Use appropriate metrics and callbacks as needed.

In [ ]:
# Training
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Callback functions used - EarlyStopping and ModelCheckpoint
# Reference: https://keras.io/api/callbacks/early_stopping/
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Reference: https://keras.io/api/callbacks/model_checkpoint/
checkpoint_filepath = '/tmp/ckpt/checkpoint.weights.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Introducing a new callback funciton after the first observation - ReduceLROnPlateau
# Reference: https://keras.io/api/callbacks/reduce_lr_on_plateau/
reduce_lr_callback = ReduceLROnPlateau(
    monitor='val_loss',    # or 'val_accuracy'
    factor=0.5,            # reduce by half
    patience=2,            # wait 2 epochs before reducing
    min_lr=1e-6,           # don't go below this
    verbose=1              # logs when LR is reduced
)

# Reference: https://keras.io/api/models/model_training_apis/#fit-method
history = model.fit(
    X_train, 
    encoded_labels, 
    batch_size=32, 
    epochs=10, 
    validation_split=0.3,
    callbacks=[early_stopping_callback, model_checkpoint_callback, reduce_lr_callback]
)

# Quick Observations - Iteration 1:
#----------------------------------
# Training Accuracy is imploving by a small margin
# Loss is gradually decreasing
# Validation loss is huge! The training set it overfitting
#
# Epoch 1/10━━━━━━━━━━━━━━━━━━━━ - accuracy: 0.1639 - loss: 2.7129 - val_accuracy: 0.0000e+00 - val_loss: 22.4157
# Epoch 2/10━━━━━━━━━━━━━━━━━━━━ - accuracy: 0.2416 - loss: 2.0073 - val_accuracy: 6.2422e-04 - val_loss: 30.6398
# Epoch 3/10━━━━━━━━━━━━━━━━━━━━ - accuracy: 0.2680 - loss: 1.9028 - val_accuracy: 6.2422e-04 - val_loss: 35.1025
# Epoch 4/10━━━━━━━━━━━━━━━━━━━━ - accuracy: 0.2704 - loss: 1.7594 - val_accuracy: 0.0062 - val_loss: 59.8457
#----------------------------------

# Quick Observations - Iteration 2:
#----------------------------------
# Overfitting has reduced
# Learning rate reduction helped in reducing the validation loss
# ReduceLROnPlateau kicked in after 
#----------------------------------

### **3.2 Model Testing and Evaluation** <font color=red> [5 marks] </font><br>

#### **3.2.1** <font color=red> [5 marks] </font><br>
Evaluate the model on test dataset. Derive appropriate metrics.

In [ ]:
# Evaluate on the test set; display suitable metrics
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)

print(np.unique(np.argmax(y_pred, axis=1), return_counts=True))

In [ ]:
print(y_pred_classes)
print(y_val)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_val, y_pred_classes)

class_names = label_encoder.classes_

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names,
            annot_kws={"size": 10}) 

plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.title('Confusion Matrix', fontsize=14)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

cm_df = pd.DataFrame(cm, index=[f"True: {label}" for label in class_names],
                        columns=[f"Pred: {label}" for label in class_names])

print("Confusion Matrix:\n")
print(cm_df.to_string())


## **4. Data Augmentation** <font color=red> [optional] </font><br>

#### **4.1 Create a Data Augmentation Pipeline**

##### **4.1.1**
Define augmentation steps for the datasets.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1,
    rescale=1./255,
    validation_split=0.3
)

Augment and resample the images.
In case of class imbalance, you can also perform adequate undersampling on the majority class and augment those images to ensure consistency in the input datasets for both classes.

Augment the images.

In [ ]:
# Create a function to augment the images


In [ ]:
# Create the augmented training dataset
train_augmented_data = train_datagen.flow(
    X_train, 
    y_train,
    batch_size=32,
    subset='training'
)

val_augmented_data = train_datagen.flow(
    X_train, 
    y_train,
    batch_size=32,
    subset='validation'
)

##### **4.1.2**

Train the model on the new augmented dataset.

In [ ]:
# Train the model using augmented images
history_augmented = model.fit(
    train_augmented_data,
    epochs=10,
    validation_data=val_augmented_data,
    callbacks=[early_stopping_callback, model_checkpoint_callback, reduce_lr_callback]
)

In [ ]:
from tensorflow.keras.preprocessing import image

img_path = os.path.join('waste_dataset', 'data', 'Cardboard', 'file_12.png')

img = image.load_img(img_path, target_size=(64,64))
img_array = image.img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)[0]
class_names = label_encoder.classes_

print("Prediction probabilities:")
for i, prob in enumerate(predictions):
    print(f"{class_names[i]}: {prob * 100:.2f}%")

plt.imshow(img)
plt.axis('off')
plt.title("Test Image")
plt.show()

## **5. Conclusions** <font color = red> [5 marks]</font>

#### **5.1 Conclude with outcomes and insights gained** <font color =red> [5 marks] </font>

* Report your findings about the data
* Report model training results